In [1]:
import sklearn 
from sklearn.cluster import KMeans, AgglomerativeClustering, MeanShift, estimate_bandwidth, DBSCAN
from sklearn_extra.cluster import KMedoids

from sklearn.neighbors import   NearestCentroid, \
                                KNeighborsClassifier, \
                                KernelDensity
from sklearn.mixture import GaussianMixture

from sklearn.metrics import silhouette_samples, adjusted_rand_score, \
                            mutual_info_score, normalized_mutual_info_score, \
                            adjusted_mutual_info_score,  \
                            homogeneity_score, completeness_score, v_measure_score, \
                            fowlkes_mallows_score,silhouette_score, calinski_harabasz_score
import numpy as np
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster

In [2]:
# Importamos los datos solo con las características más importantes

X_reduc=np.load("./datos_mfcc_pca99_X_reducido_kmeans.npy")
X_reduc2=np.load("./datos_mfcc_pca99_X_reducido_2kmeans.npy")
y_reduc=np.load("./datos_mfcc_pca99_y_reducido.npy")
print(X_reduc.shape)
print(X_reduc2.shape)
print(y_reduc.shape)

(360, 464)
(360, 5)
(360,)


In [3]:
# MEANSHIFT  SIN REDUCCION DE DIMENSIONALIDAD

# This way of estimating bandwith is part of the MeanShift algorithm
bandwidth = estimate_bandwidth(X_reduc, quantile=0.2,random_state=0)
print("Estimated bandwidth=%.2f"%bandwidth)

Estimated bandwidth=787.48


In [4]:
ms = MeanShift(bandwidth=bandwidth, bin_seeding=True, 
               cluster_all=True).fit(X_reduc)

labels = ms.labels_
cluster_centers = ms.cluster_centers_
labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique) - (1 if -1 in labels else 0)

print('Bandwidth: ' , bandwidth)
print("number of estimated clusters : %d" % n_clusters_)
print('Labels: ' , set(labels))

Bandwidth:  787.4817708333334
number of estimated clusters : 2
Labels:  {0, 1}


Vemos que sin la reducción de la dimensionalidad solamente me detecta dos clusters. No es un resultado muy bueno ya que sabemos que en realidad debería haber seis clusters, cada uno de ellos correspondiendo a cada una de las clases.

In [5]:
# MEANSHIFT  CON REDUCCION DE DIMENSIONALIDAD

# This way of estimating bandwith is part of the MeanShift algorithm
bandwidth = estimate_bandwidth(X_reduc2, quantile=0.2,random_state=0)
print("Estimated bandwidth=%.2f"%bandwidth)

Estimated bandwidth=358.46


In [6]:
ms = MeanShift(bandwidth=bandwidth, bin_seeding=True, 
               cluster_all=True).fit(X_reduc2)

labels = ms.labels_
cluster_centers = ms.cluster_centers_
labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique) - (1 if -1 in labels else 0)

In [7]:
print('Bandwidth: ' , bandwidth)
print("number of estimated clusters : %d" % n_clusters_)
print('Labels: ' , set(labels))

Bandwidth:  358.46366281834366
number of estimated clusters : 4
Labels:  {0, 1, 2, 3}


Según el algoritmo de clustering MeanShift (sobre los datos reducidos), tendríamos cuatro posibles clusters. 

Nosotros sabemos que en realidad en nuestros datos debería haber seis clusters porque tenemos seis posibles clases. El hecho de que MeanShift nos dé solo cuatro clusters se puede deber a que éstos estén solapados entre sí. Como este algoritmo funciona por densidades, es facil que considere que dos clusters solapados sean uno en vez de dos.

Se nota una mejoría al realizar el clustering sobre un conjunto de muestras de menor dimensionalidad.

Podemos probar a variar el parámetro `bandwith` para ver si obtenemos el resultado que nos gustaría a nosotros. 

In [11]:
ms = MeanShift(bandwidth=325, bin_seeding=True, 
               cluster_all=True).fit(X_reduc2)

labels = ms.labels_
cluster_centers = ms.cluster_centers_
labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique) - (1 if -1 in labels else 0)

print('Bandwidth: ' , bandwidth)
print("number of estimated clusters : %d" % n_clusters_)
print('Labels: ' , set(labels))

Bandwidth:  358.46366281834366
number of estimated clusters : 6
Labels:  {0, 1, 2, 3, 4, 5}


Por ejemplo, reduciendo un poco el valor de `bandwith` obtenemos el resultado que nos gustaría a nosotros, seis clusters. 

Se podrían calcular algunas métricas para ver cómo de buenos son los clusters, pero para ello necesitaríamos las etiquetas. Además, ese no es el objetivo del primer apartado sino que el objetivo más bien es una primera aproximación a los datos para intentar descubrir posibles patrones.